In [100]:
import numpy as np 
import pandas as pd 


In [102]:
timesteps = 14
startDay = 350


In [104]:
dt = pd.read_csv("sales_train_validation.csv")
dt.head(3)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1


In [105]:
print(dt.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB
None


In [106]:
def reduction_mem(df):
    float_cols = [c for c in df if df[c].dtype == 'float64']
    int_cols = [c for c in df if df[c].dtype in ['int64', 'int32']]
    df[float_cols] = df[ float_cols].astype(np.float16)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df
    


In [107]:
dt = reduction_mem(dt)


In [108]:
print(dt.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int16(1913), object(6)
memory usage: 112.6+ MB
None


In [109]:
dt = dt.T


In [110]:
dt = dt[6 + startDay:]
dt.head(5)


,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_351,0,0,0,2,0,0,0,24,3,2,...,0,9,1,0,11,0,0,1,0,0
d_352,0,0,0,0,0,0,0,9,0,2,...,0,5,4,0,8,0,1,2,0,0
d_353,0,0,0,4,2,0,0,2,1,1,...,0,15,2,0,3,0,1,2,0,0
d_354,0,1,0,2,0,0,0,7,1,0,...,0,5,1,0,3,0,0,0,0,0
d_355,0,0,0,1,2,0,0,0,0,0,...,0,7,1,0,1,0,1,1,0,0


In [111]:
cal_data = pd.read_csv( 'calendar.csv')


In [113]:
daysBeforeEvent = pd.DataFrame(np.zeros((1969,1)))


In [122]:
for x,y in cal_data.iterrows():
    if((pd.isnull(cal_data["event_name_1"][x])) == False):
           daysBeforeEvent[0][x-1] = 1 


/var/folders/h3/lplmwndn2l1cw5q526nm0rlh0000gn/T/ipykernel_32651/452492889.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  daysBeforeEvent[0][x-1] = 1


In [124]:
del cal_data


In [126]:
daysBeforeEventTest = daysBeforeEvent[1913:1941]
#"daysBeforeEvent" will be used for training as a feature.
daysBeforeEvent = daysBeforeEvent[startDay:1913]


In [128]:
daysBeforeEvent.columns = ["oneDayBeforeEvent"]
daysBeforeEvent.index = dt.index


In [130]:
dt = pd.concat([dt, daysBeforeEvent], axis = 1)

dt.columns


Index([                  0,                   1,                   2,
                         3,                   4,                   5,
                         6,                   7,                   8,
                         9,
       ...
                     30481,               30482,               30483,
                     30484,               30485,               30486,
                     30487,               30488,               30489,
       'oneDayBeforeEvent'],
      dtype='object', length=30491)

In [132]:
import numpy as np

# Updated Min-Max Scaler Implementation
class MinMaxScalerCustom:
    def __init__(self, feature_range=(0, 1)):
        self.min_ = None
        self.max_ = None
        self.scale_ = None
        self.min_range, self.max_range = feature_range

    def fit(self, data):
        """
        Compute the min and max values for scaling.
        """
        self.min_ = np.min(data, axis=0)  # Minimum value for each feature
        self.max_ = np.max(data, axis=0)  # Maximum value for each feature
        self.scale_ = self.max_ - self.min_

    def transform(self, data):
        """
        Scale the data to the feature range (min_range, max_range).
        """
        if self.min_ is None or self.max_ is None:
            raise ValueError("The scaler has not been fitted yet.")
        
        # Avoid division by zero
        scale_adjusted = np.where(self.scale_ == 0, 1, self.scale_)
        
        # Scale data to 0-1
        data_scaled = (data - self.min_) / scale_adjusted
        # Scale data to the desired range
        data_scaled = data_scaled * (self.max_range - self.min_range) + self.min_range
        return data_scaled

    def fit_transform(self, data):
        """
        Fit the scaler to the data and then transform it.
        """
        self.fit(data)
        return self.transform(data)


sc = MinMaxScalerCustom(feature_range=(0, 1))
dt_scaled = sc.fit_transform(dt)

# Print the original and scaled data
print("Original Data:\n", dt)
print("Scaled Data:\n", dt_scaled)


Original Data:
         0  1  2  3  4  5  6   7  8  9  ... 30481 30482 30483 30484 30485  \
d_351   0  0  0  2  0  0  0  24  3  2  ...     9     1     0    11     0   
d_352   0  0  0  0  0  0  0   9  0  2  ...     5     4     0     8     0   
d_353   0  0  0  4  2  0  0   2  1  1  ...    15     2     0     3     0   
d_354   0  1  0  2  0  0  0   7  1  0  ...     5     1     0     3     0   
d_355   0  0  0  1  2  0  0   0  0  0  ...     7     1     0     1     0   
...    .. .. .. .. .. .. ..  .. .. ..  ...   ...   ...   ...   ...   ...   
d_1909  1  1  1  0  1  0  1   4  0  0  ...     1     1     0     0     0   
d_1910  3  0  0  1  2  0  0   6  0  0  ...     3     3     0     2     1   
d_1911  0  0  1  3  2  2  0   3  0  2  ...     1     6     0     3     0   
d_1912  1  0  1  7  2  0  1   2  0  0  ...     0     0     4     2     0   
d_1913  1  0  1  2  4  0  1   1  0  2  ...     2     1     0     1     1   

       30486 30487 30488 30489 oneDayBeforeEvent  
d_351      0     1  

In [134]:
X_train = []
y_train = []
for i in range(timesteps, 1913 - startDay):
    X_train.append(dt_scaled[i-timesteps:i])
    y_train.append(dt_scaled[i][0:30490]) 


In [136]:
del dt_scaled


In [138]:
#Convert to np array to be able to feed the LSTM model
X_train = np.array(X_train, dtype = 'float16')
y_train = np.array(y_train, dtype = 'float16')
print(X_train.shape)
print(y_train.shape)


(1549, 14, 30491)
(1549, 1563)


In [152]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Define the BiLSTM Model
class BiLSTMModel(nn.Module):
    def __init__(self, input_size, sequence_length, output_size):
        super(BiLSTMModel, self).__init__()
        self.sequence_length = sequence_length
        
        # Bidirectional LSTM Layer 1
        self.lstm1 = nn.LSTM(input_size, 64, bidirectional=True, batch_first=True)
        self.dropout1 = nn.Dropout(0.25)
        
        # Bidirectional LSTM Layer 2
        self.lstm2 = nn.LSTM(64 * 2, 16, bidirectional=True, batch_first=True)
        
        # Fully connected output layer (adjusted to match target shape)
        self.fc = nn.Linear(16 * 2, 1563)  # Adjusted to match target shape

    def forward(self, x):
        # First LSTM Layer
        x, _ = self.lstm1(x)
        x = self.dropout1(x)
        
        # Second LSTM Layer (get last output from the sequence)
        x, _ = self.lstm2(x)
        
        # Only use the last output from the sequence
        x = x[:, -1, :]  # Shape: (batch_size, 16 * 2)
        
        # Fully connected output layer
        x = self.fc(x)
        return x


# Generate some example data (replace with your actual dataset)
# For illustration, we'll use random data for X_train and y_train.
X_train = np.random.randn(1000, 10, 20)  # Example shape (batch_size, sequence_length, input_size)
y_train = np.random.randn(1000, 1563)    # Example target shape (batch_size, output_size)

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

# Define the model, loss function, and optimizer
input_size = X_train.shape[2]   # Input feature size (20)
sequence_length = X_train.shape[1]  # Sequence length (10)
output_size = 1563  # Target size (1563)

model = BiLSTMModel(input_size, sequence_length, output_size)
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(batch_X)
        
        # Compute the loss
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

# Save the trained model (optional)
# torch.save(model.state_dict(), 'bilstm_model.pth')

# Now you can test or use the model for predictions
model.eval()
with torch.no_grad():
    sample_input = torch.tensor(np.random.randn(1, 10, 20), dtype=torch.float32)
    prediction = model(sample_input)
    print("Prediction:", prediction)


Epoch 1/20, Loss: 1.007039536833763
Epoch 2/20, Loss: 1.0006005787849426
Epoch 3/20, Loss: 0.9994258749485015
Epoch 4/20, Loss: 0.9987697786092758
Epoch 5/20, Loss: 0.9978496676683426
Epoch 6/20, Loss: 0.9964442044496536
Epoch 7/20, Loss: 0.9948855066299438
Epoch 8/20, Loss: 0.9928825354576111
Epoch 9/20, Loss: 0.9907584106922149
Epoch 10/20, Loss: 0.9887301462888718
Epoch 11/20, Loss: 0.9865404093265533
Epoch 12/20, Loss: 0.9845842534303665
Epoch 13/20, Loss: 0.9823258292675018
Epoch 14/20, Loss: 0.9804198235273361
Epoch 15/20, Loss: 0.9787947487831116
Epoch 16/20, Loss: 0.9769445097446442
Epoch 17/20, Loss: 0.9753510880470276
Epoch 18/20, Loss: 0.9737490236759185
Epoch 19/20, Loss: 0.9724401897192001
Epoch 20/20, Loss: 0.9711301320791245
Prediction: tensor([[-0.0995, -0.0555,  0.0253,  ...,  0.0502,  0.2157, -0.1147]])


In [154]:
inputs= dt[-timesteps:]
inputs = sc.transform(inputs)
